In [1]:
import numpy as np
import pandas as pd
import torch
import sklearn
import matplotlib.pyplot as plt
import sys
import os

## Raincouver







Markov chain:

Fit per year

Fit per month

Fit per season

Fit across all data

Time inhomogeneous per day (data permitting)

Maybe higher order chain


HMM:

Across all data
Across year

Implement baum welch
Implement forward backward

overkill:

VRNN:

Same stuff


Transformer:

Same stuff


Maybe do some viz 
Would be nice to  get a t-SNE decomposition that shows sort of seasons


More rich feature representations



https://climate.weather.gc.ca/historical_data/search_historic_data_e.html





## Data Prep


In [6]:
%%sh
cd /Users/alansmacbook/Desktop/Learning/Raincouver/data;
for year in `seq 1926 2021`; do
    if ! [ -e "en_climate_daily_BC_1108446_${year}_P1D.csv" ]; then
        wget --content-disposition "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=888&Year=${year}&timeframe=2&submit= Download+Data";
    fi;
done;

In [165]:
data_dir = "/Users/alansmacbook/Desktop/Learning/Raincouver/data/"
csv_files = os.listdir(data_dir)

frames = []
for file in csv_files:
    frames.append(pd.read_csv(data_dir + file))

rain_data = pd.concat(frames)

rain_data.columns
rain_data = rain_data.drop(columns=
                ['Longitude (x)', 
                'Latitude (y)', 
                'Station Name', 
                'Climate ID',
                'Max Temp Flag', 
                'Min Temp Flag',
                'Mean Temp Flag', 
                'Heat Deg Days (°C)', 
                'Heat Deg Days Flag',
                'Cool Deg Days (°C)',
                'Cool Deg Days Flag',
                'Snow on Grnd (cm)',
                'Snow on Grnd Flag', 
                'Dir of Max Gust (10s deg)',
                'Dir of Max Gust Flag', 
                'Spd of Max Gust (km/h)',
                'Spd of Max Gust Flag'])

rain_data = rain_data.sort_values("Date/Time").reset_index(drop=True)

rain_data.to_csv("all.csv")





In [349]:
# test_mat = np.array([[1,0,1],
#                     [0,1,0],
#                     [1,1,0]])
test_mat = np.array([[1,1],
                     [0,1],
                     [1,0],
                     [0,0]])
chain  = MarkovChain(test_mat)
chain.sample(10)
# print(chain.P)
# print(chain.pi)
# print(chain.marginals(10, method='exact'))
# print(chain.marginals(10, method='Monte Carlo'))

S, U = np.linalg.eig(chain.P.T)
stationary = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat)
stationary = stationary / np.sum(stationary)
# print(stationary)
# print(chain.backwardConditionals(1, 10,0, method='exact'))
# print(chain.backwardConditionals(1, 10,0, method='Monte Carlo', nsamples=10000))





[[0.5 0.5]
 [0.5 0.5]]
[0.5 0.5]


In [330]:

class MarkovChain():
    
    def __init__(self, X, d_state=2):
        self.P, self.pi = self.fit(X,d_state)
        self.d = d_state

        
    def fit(self, X, d_state):
        n,t = np.shape(X)
        P = np.zeros((d_state,d_state), dtype=np.float64)
        pi = np.bincount(X[:,0])/n
        transitions = np.bincount(X[:,0:t-1].flatten())
        for i in range(d_state):
            for j in range(d_state):
                P[i,j] = np.sum([np.sum((X[:,t] == i) & (X[:,t+1] == j)) for t in range(t-1)])
            P[i] /= transitions[i]
        return P, pi
    
    
    def sample(self, length, nsamples=1):
        samples = np.zeros((nsamples,length), dtype=np.int64)
        for i in range(nsamples):
            samples[i,0] = np.random.choice(self.d,p=self.pi)
            for j in range(1,length):
                samples[i,j] = np.random.choice(self.d,p=self.P[samples[i,j-1]])
        return samples
    
        
    def chapmanKolmogorov(self, time, initial=None):
        marginals = np.zeros((self.d,time), dtype=np.float64)
        marginals[:,0] = initial if initial is not None else self.pi
        for i in range(1,time):
            marginals[:,i] = self.P.T @ marginals[:, i - 1]
        return marginals
    
    
    def marginals(self, time, method='exact', nsamples=10000):
        if method == 'exact':
            return self.chapmanKolmogorov(time)[:,time-1]
        elif method == 'Monte Carlo':
            samples =  self.sample(time,nsamples)
            return np.bincount(samples[:,time-1])/nsamples
        else:
            raise "Method not recognized."
        
    
    def forwardConditionals(self, posterior, condition_time,  condition_value, method='exact', nsamples=10000):
        assert condition_value < self.d
        assert condition_time <= posterior
        if method == 'exact':
            one_hot = np.zeros(self.d)
            one_hot[condition_value] = 1.0
            return self.chapmanKolmogorov(posterior - condition_time + 1, initial=one_hot)[:,posterior - condition_time]
        elif method == 'Monte Carlo':
            samples = self.sample(posterior, nsamples)
            filtered_samples = samples[samples[:, condition_time - 1] == condition_value]
            print(f"{np.shape(filtered_samples)[0]/nsamples}% of {nsamples} samples were rejected.")
            return np.bincount(filtered_samples[:,posterior - 1])/np.shape(filtered_samples)[0]
        else:
            raise "Method not recognized."

    
    def backwardConditionals(self, posterior, condition_time,  condition_value, method='exact', nsamples=10000):
        assert condition_value < self.d
        assert condition_time >= posterior
        if method == 'exact':
            M = self.chapmanKolmogorov(condition_time)
            V = np.zeros_like(M)
            V[condition_value, condition_time - 1] = 1.0
            for i in range(condition_time - 1, 0, -1):
                V[:,i-1] = self.P @ V[:,i]
            MV = M*V
            return MV[:, posterior -  1]/np.sum(MV[:, posterior -  1])
        
        elif method == 'Monte Carlo':
            samples = self.sample(condition_time, nsamples)
            filtered_samples = samples[samples[:, condition_time - 1] == condition_value]
            print(f"{np.shape(filtered_samples)[0]/nsamples}% of {nsamples} samples were rejected.")
            return np.bincount(filtered_samples[:,posterior - 1])/np.shape(filtered_samples)[0]
        else:
            raise "Method not recognized."

    
    def decode(self, time):
        M = np.zeros((self.d,time), dtype=np.float64)
        B = np.zeros((self.d,time), dtype=np.int64)
        M[:,0] = self.pi
        for i in range(1,time):
            for j in range(self.d):
                transition = [M[k,i-1]*self.P[k,j] for k in range(self.d)]
                M[j,i] = np.max(transition)
                B[j,i] = np.argmax(transition)
        prob = np.max(M[:,time-1])
        decoding = np.zeros(time, dtype=np.int64)
        decoding[time-1] = np.argmax(M[:,time-1])
        for i in range(time - 1, 0, -1):
            decoding[i-1] = B[decoding[i], i]
        return decoding, prob
    
    def score(self, chain):
        log_prob = 0.0
        log_prob -= np.log(self.pi[chain[0]])
        for i in range(1,np.size(chain)):
            log_prob -= np.log(self.P[chain[i - 1],chain[i]])
        return log_prob
        
    

In [343]:
class HMM():
    
    def __init__(self, X, d_state=2, d_hidden=4):
        self.P, self.O, self.pi = self.fit(X)
        self.d = d_state
        self.h = d_hidden
    
    # Baum-Welch
    def fit(self, X):
        pass
    
    def sample(self, length, nsamples=1):
        samples = np.zeros((nsamples,length), dtype=np.int64)
        for i in range(nsamples):
            z = np.random.choice(self.h,p=self.pi)
            samples[i,0] = np.random.choice(self.d, p=self.O[z])
            for j in range(1,length):
                z = np.random.choice(self.d,p=self.P[z])
                samples[i,j] = np.random.choice(self.d, p=self.O[z])
        return samples
        
    
    def marginals(self, time, method='exact'):
        pass

    def score(self, chain):
        pass


    

In [50]:

# hmmmm
class VRNN(torch.nn.Module):
    
    def __init__(self):
        pass
    
    def fit(self):
        pass
    
    def sample(self):
        pass
    
    def predict(self):
        pass